In [ ]:
import numpy as np
import scipy.sparse as sparse
import scipy.sparse.linalg as linalg


class Element(object):
    def __init__(self, node_a, node_b):
        self.node_a = node_a
        self.node_b = node_b
        
    def get_dof(self):
        return (self.node_a.dof_a, self.node_a.dof_b,
                self.node_b.dof_a, self.node_b.dof_b)
    
    def get_boundary(self):
        return (self.node_a.boundary, self.node_a.boundary,
                self.node_b.boundary, self.node_b.boundary)
        
        
class Node(object):
    def __init__(self, coord):
        self.coord = coord
        self.boundary = False
        self.dof_a = 0
        self.dof_b = 0
        
        
def mesh(xa, xb, n_elements): 
    n_nodes = n_elements + 1  
    xcoords, step = np.linspace(xa, xb, num=n_nodes, retstep=True) 
    nodes = [Node(x) for x in xcoords] 
    nodes[0].boundary = True     
    elements = [Element(nodes[i], nodes[i + 1]) for i in range(n_elements)] 

    a = 0.5 * step
    n_dof = 2 * (n_nodes - 1)
    
    dof_count = 0 
    for n in nodes:
        if not n.boundary:
            n.dof_a = dof_count
            n.dof_b = dof_count + 1
            dof_count += 2

    return elements, a, n_dof
        
    
def shapes(xi, a):
    n = [0.5 * (1 - xi), 0.5 * (1 + xi)]
    dn = [-0.5 / a, 0.5 / a]
    return n, dn


def element_matrices(a, G, E, Iz, rho, A, kappa):
    ke = np.zeros((4, 4))
    me = np.zeros((4, 4))

    points = [0.577350269189626, -0.577350269189626]
    for p in points:
        n, dn = shapes(p, a)
        Ba = np.array([[0, dn[0], 0, dn[1]]])
        Bb = np.array([[n[0], 0, n[1], 0]])
        Bc = np.array([[0, n[0], 0, n[1]]])

        ke += E * Iz * Ba.T @ Ba * a
        me += a * rho * A * Bb.T @ Bb + a * rho * Iz * Bc.T @ Bc

    n, dn = shapes(0.0, a)
    Bd = np.array([[dn[0], n[0], dn[1], n[1]]])
    ke += 2 * a * kappa * G * A * Bd.T @ Bd
    
    return ke, me


def assemble(elements, ke, me, n_dof): 
    rows, cols, kk, mm = [], [], [], []

    for e in elements:
        dof = e.get_dof() 
        boundary = e.get_boundary() 

        for i in range(4):
            for j in range(4):
                if not boundary[i] and not boundary[j]:
                    rows.append(dof[i])
                    cols.append(dof[j])
                    kk.append(ke[i, j])
                    mm.append(me[i, j])

    K = sparse.coo_matrix((kk, (rows, cols)), shape=(n_dof, n_dof)).tocsr()
    M = sparse.coo_matrix((mm, (rows, cols)), shape=(n_dof, n_dof)).tocsr()

    return K, M


def modal_analysis(n_elements, n_modes): 
    L = 5.0
    E = 200e9
    v = 0.3
    rho = 7850
    A = 0.25
    I = 1/192
    kappa = 5/6
    G = E / (2 * (1 + v))
    h = 0.5
    wid = 0.5
    Iz = wid * h**3 / 12

    elements, a, n_dof = mesh(0, L, n_elements)
    ke, me = element_matrices(a, G, E, Iz, rho, A, kappa)
    K, M = assemble(elements, ke, me, n_dof)

    w2, _ = linalg.eigsh(K, k=n_modes, M=M, sigma=0, which='LM') 
    omegas = np.sqrt(w2)

    return omegas / (2 * np.pi) 


if __name__ == "__main__":
    freqs = modal_analysis(300, 5)
    print("FEM (Timoshenko) 悬臂梁模态频率：")
    for i, f in enumerate(freqs, 1):
        print(f"第 {i} 阶: {f:.4f} Hz")
